In [9]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pandas.testing import assert_frame_equal
#from pysankey2 import Sankey
from scipy import stats
#from pysankey2.datasets import load_fruits
import os

In [10]:
sc.set_figure_params(dpi = 300, color_map = 'viridis_r' )
sc.settings.verbosity = 2

In [11]:
#%load_ext rpy2.ipython

# inferCNV Pseudo Neighbor

In [20]:
# Presets

patient_id = 'Glioma_p2'

# Automatically assign cancer_type

if ('HCC' in patient_id) or ('ICC' in patient_id):

    cancer_type = 'HCC_ICC'

elif ('ESCA' in patient_id):

    cancer_type = 'ESCA'

elif ('LUAD' in patient_id):

    cancer_type = 'LUAD'

elif ('COAD' in patient_id) or ('READ' in patient_id):

    cancer_type = 'COAD_READ'

elif ('BRCA' in patient_id):

    cancer_type = 'BRCA'

elif ('STAD' in patient_id):

    cancer_type = 'STAD'

elif ('Glioma' in patient_id):

    cancer_type = 'Glioma'


#gene_loc = pd.read_csv('/mnt/hdd1/jikai/cnv_test/gene_loc.tsv', sep = '\t', header=None)

In [21]:
# Loading Clone

# Loading inferCNV clone
infer_cnv_clone = pd.read_csv('/mnt/hdd1/jikai/hhseq_processing_latest/inferCNV_per_cancer_type/' + cancer_type 
                              + '_all/inferCNV/' + patient_id + 
                              '_default_test/pyramidinal_101/infercnv_reshape.observation_groupings.txt', sep = ' ')

# Loading Numbat Clone
#cnv_clone = pd.read_csv('/mnt/hdd1/jikai/numbat/' + patient_id + '/' + patient_id + '_cnv_clone_sorted.csv', 
#                        index_col = 0)

In [22]:
# Loading CNV Score

respath = "/mnt/hdd1/jikai/hhseq_processing_latest/inferCNV_per_cancer_type/" + cancer_type + "_all/inferCNV/" + patient_id + "_default_test/pyramidinal_101/"

# Loading CNV Score: Raw
cnv_score_ob = pd.read_csv(respath + "infercnv_reshape.observations.txt",sep=' ')
cnv_score_ob = cnv_score_ob.T.copy()

# Loading CNV Score: bin
#cnv_score_ob_cut = pd.read_csv(respath + 'cnv_score_ob_cut.csv', index_col=0)

In [23]:
# Loading adata
adata = sc.read('/mnt/hdd1/jikai/hhseq_processing_latest/annotation_per_patient/' + patient_id + '/' + 
                patient_id + '_annotation.h5ad')

In [24]:
'''
https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.neighbors.html

connectivitiessparse matrix of dtype float32.
Weighted adjacency matrix of the neighborhood graph of data points. Weights should be interpreted as connectivities.

distancessparse matrix of dtype float32.
Instead of decaying weights, this stores distances for each pair of neighbors.
'''
neighbor_df = adata.obsp['connectivities']
neighbor_df = pd.DataFrame(neighbor_df.todense())
neighbor_df.index = adata.obs.index.tolist()
neighbor_df.columns = adata.obs.index.tolist()

In [25]:
# LUAD_p4 Only: Remove Mast Cell Cluster
#len(infer_cnv_clone.index.tolist())
#cell_retain = list(set(infer_cnv_clone.index.tolist()).difference(set(adata[adata.obs['Celltype'] == 'Mast cell'].obs.index.tolist())))
#len(cell_retain)

#cnv_score_ob = cnv_score_ob.loc[cell_retain, :]
#infer_cnv_clone = infer_cnv_clone.loc[cell_retain, :]

In [26]:
# Additional Settings
n_pseudo_cell = 9
cnv_score_used = cnv_score_ob.copy()
cnv_clone_used = infer_cnv_clone.copy()
clone_column = 'Dendrogram Group'

In [27]:
cnv_score_neighbor_all = pd.DataFrame(index=cnv_score_used.columns.tolist())
neighbor_df = neighbor_df.loc[cnv_score_used.index.tolist(), cnv_score_used.index.tolist()]

for each_cell in tqdm(cnv_score_used.index.tolist()):
    
    each_cell_neighbor_subset = neighbor_df[[each_cell]]
    each_cell_neighbor_subset = each_cell_neighbor_subset[each_cell_neighbor_subset[each_cell] != 0]
    each_cell_neighbor_subset = pd.DataFrame(each_cell_neighbor_subset[each_cell].sort_values(ascending = False))
    each_cell_neighbor_subset = each_cell_neighbor_subset[:n_pseudo_cell-1]
    
    neighbor_cell_list = each_cell_neighbor_subset.index.tolist()
    neighbor_cell_list.append(each_cell)
    cnv_score_subset = cnv_score_used.loc[neighbor_cell_list, :]
    cnv_score_subset_neighbor = pd.DataFrame(cnv_score_subset.mean())
    cnv_score_subset_neighbor.columns = [each_cell]
    cnv_score_subset_neighbor = cnv_score_subset_neighbor.loc[cnv_score_neighbor_all.index.tolist(), :]
    cnv_score_neighbor_all[each_cell] = cnv_score_subset_neighbor[each_cell]
    
cnv_score_neighbor_all = cnv_score_neighbor_all.T

  1%|▎                                       | 97/14951 [00:01<04:11, 59.06it/s]/tmp/ipykernel_462284/216623671.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  cnv_score_neighbor_all[each_cell] = cnv_score_subset_neighbor[each_cell]
100%|█████████████████████████████████████| 14951/14951 [04:24<00:00, 56.59it/s]


# CNV Score (Bin): Each Chr

In [1]:
#
#HCC_p1: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#HCC_p2: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#HCC_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#HCC_p4: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#HCC_p5: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#ICC_p1: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#ICC_p2: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#ICC_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#ICC_p4: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#COAD_p1: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#COAD_p2: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#READ_p1: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2]
#
#ESCA_p1: 1.2 & bins=[0.8,0.9,0.96,1.04,1.1,1.2], labels=[2,1,0,1,2]
#ESCA_p2: NAT & No Epi Cluster
#ESCA_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#LUAD_p1: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2]
#LUAD_p2: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2]
#LUAD_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#LUAD_p4: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2] (Removing Mast Cell in Observation!!!)
#LUAD_p5: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#LUAD_p6: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#LUAD_p7: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#STAD_p1: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#BRCA_p1: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2]
#BRCA_p2: 1.1 & bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2]
#BRCA_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#BRCA_p4: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#Glioma_p1: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#Glioma_p2: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#Glioma_p3: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#Glioma_p4: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#Glioma_p5: 1.15 & bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2]
#
#

In [29]:
pd.DataFrame(cnv_score_neighbor_all.max()).max()

0    1.120332
dtype: float64

In [30]:
pd.DataFrame(cnv_score_neighbor_all.min()).min()

0    0.889048
dtype: float64

In [31]:
cnv_score_neighbor_cut = cnv_score_neighbor_all.copy()

for i in tqdm(range(len(cnv_score_neighbor_cut.columns))):
    cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]] = pd.cut(cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]], bins=[0.85,0.925,0.97,1.03,1.075,1.15], labels=[2,1,0,1,2], ordered=False)
#    cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]] = pd.cut(cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]], bins=[0.9,0.95,0.98,1.02,1.05,1.1], labels=[2,1,0,1,2], ordered=False)
#    cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]] = pd.cut(cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]], bins=[0.8,0.9,0.96,1.04,1.1,1.2], labels=[2,1,0,1,2], ordered=False)
#    cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]] = pd.cut(cnv_score_neighbor_cut[cnv_score_neighbor_cut.columns[i]], bins=[0.7,0.85,0.94,1.06,1.15,1.3], labels=[2,1,0,1,2], ordered=False)

100%|███████████████████████████████████████| 8058/8058 [26:50<00:00,  5.00it/s]


In [1]:
cnv_score_neighbor_cut_mean = pd.DataFrame(cnv_score_neighbor_cut.mean(axis = 1))

cnv_score_neighbor_cut_mean = cnv_score_neighbor_cut_mean.loc[cnv_clone_used.index.tolist(), :]
cnv_score_neighbor_cut_mean['clone'] = cnv_clone_used[clone_column]

cnv_score_neighbor_cut_mean['clone'] = cnv_score_neighbor_cut_mean['clone'].astype(str)
cnv_score_neighbor_cut_mean.to_csv(patient_id + '_cnv_score_neighbor_cut_mean.csv')

#fig, axs = plt.subplots(1, 1, figsize=(12, 12), dpi=300, sharey=True)
#sns.boxplot(data=cnv_score_neighbor_cut_mean, x='clone', y=0, fliersize=0)
#plt.xlabel('Clone')
#plt.ylabel('CNV Score: Mean')
#plt.title(patient_id)
#plt.savefig(patient_id + '_boxplot.png')
#
#plt.figure(figsize=(12,6), dpi=300)
#sns.histplot(data=cnv_score_neighbor_cut_mean, x=0, hue='clone')
#plt.xlabel('CNV Score: Mean')
#plt.title(patient_id)
#plt.savefig(patient_id + '_histplot.png')